building chatgpt from scratch with andrej karpathy

loading data

In [1]:
with open("input.txt",'r') as f:
    text = f.read()

In [2]:
print(text[:100])
print(f"the of dataset in characters: {len(text)}")

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
the of dataset in characters: 1115390


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [4]:
stoi = {ch:i for i,ch in enumerate(chars)}
print(stoi)
itoc = {i:ch for i,ch in enumerate(chars)}
print(itoc)
encode = lambda s:[stoi[c] for c in s]
decode = lambda l:''.join([itoc[i] for i in l])


{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i',

In [5]:
print(encode("preetham"))
print(decode(encode("preetham")))   # decode([54, 56, 43, 43, 58, 46, 39, 51])


[54, 56, 43, 43, 58, 46, 39, 51]
preetham


In [6]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape,data.dtype)
data[:500]

torch.Size([1115390]) torch.int64


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [7]:
n = int(0.9 * len(data))
print(n)
train_data = data[:n]
val_data = data[n:]
print(len(train_data))
print(len(val_data))

1003851
1003851
111539


In [8]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]  # these are the inputs
    target = y[t]   #these are the outputs
    print(f"when the input is {context} the output is {target}")   #we are just predicting the next letter{token} but the thing is while we are training, we train lots of data that it simultaneous understands the underlying the pattern

when the input is tensor([18]) the output is 47
when the input is tensor([18, 47]) the output is 56
when the input is tensor([18, 47, 56]) the output is 57
when the input is tensor([18, 47, 56, 57]) the output is 58
when the input is tensor([18, 47, 56, 57, 58]) the output is 1
when the input is tensor([18, 47, 56, 57, 58,  1]) the output is 15
when the input is tensor([18, 47, 56, 57, 58,  1, 15]) the output is 47
when the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the output is 58


In [ ]:
torch.manual_seed(1337)
batch_size = 4  # how many number of sequences we process parallelly [parallel processing]
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    print(data[:100])
    print(len(data))
    ix =torch.randint(len(data) - block_size,(batch_size,))  # we have done this to invalid targets len(data) - block_size and second arguement is size the first is higher limit of randoms integer sequences
    x = 
    print(ix)

get_batch("test")


tensor([ 0,  0, 19, 30, 17, 25, 21, 27, 10,  0, 19, 53, 53, 42,  1, 51, 53, 56,
        56, 53, 61,  6,  1, 52, 43, 47, 45, 46, 40, 53, 59, 56,  1, 14, 39, 54,
        58, 47, 57, 58, 39,  8,  0,  0, 14, 13, 28, 32, 21, 31, 32, 13, 10,  0,
        19, 53, 53, 42,  1, 51, 53, 56, 56, 53, 61,  6,  1, 52, 43, 47, 45, 46,
        40, 53, 59, 56,  1, 19, 56, 43, 51, 47, 53,  8,  0, 19, 53, 42,  1, 57,
        39, 60, 43,  1, 63, 53, 59,  6,  1, 45])
111539
tensor([39625, 60322, 88082, 64234])
